# Datenaufbereitung

### Reinigung der CSV-Dateien

In diesem Block werden die CSV-Dateien von der LUBW bereinigt und in ein weiterverarbeitbares Format überführt. Dies beinhaltet das Entfernen doppelter Anführungszeichen, da die Dateien andernfalls nicht korrekt ausgelesen werden können. 
1. **def find_header_row**: Über den tatsächlich relevanten Einträgen (Tabellenkopf) befindet sich in diesen Dateien ein Block mit Metadaten. Dieser wird übersprungen, damit mit den Messwerten gearbeitet werden kann.
2. **def maybe_strip_outer quotes**: Enternt die doppelten Anführungszeichen und ersetzt sie durch einfache.
3. **def clean_every_file**: führt die vorherigen Funktionen aus und speichert die bereinigten Dateien jeweils mit der Endung "_cleaned.csv" ohne Metadaten, sodass sie für die weitere Verarbeitung genutzt werden können. 
4. Abschließend wird überprüft, ob alle Dateien erfolgreich bereinigt wurden.

In [11]:
import pandas as pd
import glob
import os
import csv
from io import StringIO
from pathlib import Path

def find_header_row(path, enc="latin1", max_lines= 50):
    with open(path, "r", encoding=enc, errors="ignore") as f:
        for i, line in enumerate(f):
            if i > max_lines:
                break
            if "Messstellennummer" in line and ("Datum" in line or "Uhrzeit" in line):
                return i
    return 0


def maybe_strip_outer_quotes(s: str) -> str:
    t = s.rstrip("\r\n")
    if t.startswith('"') and t.endswith('"') and '","' in t:
        t = t[1:-1].replace('""', '"')
    return t

def clean_every_file(path: str) -> Path | None:
    path = Path(path)

    if path.stem.endswith("_cleaned"):
        return None

    out_path = path.with_name(path.stem + "_cleaned.csv")
    header = find_header_row(path)
    
    with open(path, "r", encoding="latin1", errors="ignore") as infile, \
         open(out_path, "w", encoding="latin1", newline="") as outfile:

        for _ in range(header):
            next(infile, None)

        writer = csv.writer(outfile, delimiter=",", quotechar='"', quoting=csv.QUOTE_ALL)

        for raw in infile:
            fixed = maybe_strip_outer_quotes(raw)

            row = next(csv.reader(StringIO(fixed), delimiter=",", quotechar='"', doublequote=True))

            if not any(cell.strip() for cell in row):
                continue
            writer.writerow(row)
        return out_path
    print(f"Cleaned stored: {out_path}")


all_files=glob.glob("Datensätze/UDO-LUBW/Landespegel Hydrologie/Gewässer/*/*/*Hydrologische Landespegel.csv")
created_cleaned_files = []
for p in all_files:
    output = clean_every_file(p)
    if output:
        created_cleaned_files.append(output)
        print("Cleaned:", output)

print(f"\nReady. new created: {len(created_cleaned_files)} files (_cleaned.csv)")



Cleaned: Datensätze\UDO-LUBW\Landespegel Hydrologie\Gewässer\Ablach\Menningen Ablach\Menningen Ablach-Hydrologische Landespegel_cleaned.csv
Cleaned: Datensätze\UDO-LUBW\Landespegel Hydrologie\Gewässer\Ach\Blaubeuren Ach\Blaubeuren-Hydrologische Landespegel_cleaned.csv
Cleaned: Datensätze\UDO-LUBW\Landespegel Hydrologie\Gewässer\Acher\Kappelrodeck Acher\Kappelrodeck Acher-Hydrologische Landespegel_cleaned.csv
Cleaned: Datensätze\UDO-LUBW\Landespegel Hydrologie\Gewässer\Achkanal\Blaubeuren Achkanal\Blaubeuren Achkanal-Hydrologische Landespegel_cleaned.csv
Cleaned: Datensätze\UDO-LUBW\Landespegel Hydrologie\Gewässer\Aich\Oberensingen Aich\Oberensingen Aich-Hydrologische Landespegel_cleaned.csv
Cleaned: Datensätze\UDO-LUBW\Landespegel Hydrologie\Gewässer\Aitrach\Lauben Aitrach\Lauben Aitrach-Hydrologische Landespegel_cleaned.csv
Cleaned: Datensätze\UDO-LUBW\Landespegel Hydrologie\Gewässer\Alb\Ettlingen Alb\Ettlingen Alb-Hydrologische Landespegel_cleaned.csv
Cleaned: Datensätze\UDO-LUBW\Lan

### Konvertierung
In diesem block werden die bereinigten Dateien konvertiert. Dabei werden in der Spalte der Abflusswerte die Kommata als Dezimaltrennzeichen durch Punkte ersetzt, um Missverständnissen mit den Spaltentrennzeichen der CSV-Dateien zu vermeiden. Bisher waren die jeweiligen Einträge als Strings vorliegend. Um sie als Zahlenwerte zu definieren, werden die Einträge der Abflussspalte in Fließkommazahlen (float) konvertiert. Zusätzlich wird die Spalte "Datum / Uhrzeit" in ein Datetime-Format überführt, um zeitbasierte Auswertungen zu ermöglichen. Nach der Konvertierung werden die Dateien mit der Endung "_converted.csv" gespeichert.

In [12]:
all_cleaned_files=glob.glob("Datensätze/UDO-LUBW/Landespegel Hydrologie/Gewässer/*/*/*_cleaned.csv")
def convert_all_cleaned_files():
    created = []
    for p in all_cleaned_files:
        p = Path(p)
        df = pd.read_csv(p, sep=",", decimal=",", encoding="latin1", thousands=".")
        
        df["Wert"] = df["Wert"].astype(str).str.replace(",", ".", regex=False)
        df["Wert"] = df["Wert"].str.replace(",", ".", regex=False)
        df["Wert"] = df["Wert"].astype(float)
        # df["Wert"] = pd.to_numeric(df["Wert"], errors="coerce")
        df["Datum / Uhrzeit"] = pd.to_datetime(df["Datum / Uhrzeit"], errors="coerce")

        output = p.with_name(p.stem + "_converted.csv")
        df.to_csv(output, index=False, sep=",", encoding="utf-8", quotechar='"')

        print(f"Converted: {output}")
        created.append(output)

    return created

created_cleaned_converted_files = convert_all_cleaned_files()
        

print(f"\nReady. new created: {len(created_cleaned_converted_files)} files (_converted.csv)")

Converted: Datensätze\UDO-LUBW\Landespegel Hydrologie\Gewässer\Ablach\Menningen Ablach\Menningen Ablach-Hydrologische Landespegel_cleaned_converted.csv
Converted: Datensätze\UDO-LUBW\Landespegel Hydrologie\Gewässer\Ach\Blaubeuren Ach\Blaubeuren-Hydrologische Landespegel_cleaned_converted.csv
Converted: Datensätze\UDO-LUBW\Landespegel Hydrologie\Gewässer\Acher\Kappelrodeck Acher\Kappelrodeck Acher-Hydrologische Landespegel_cleaned_converted.csv
Converted: Datensätze\UDO-LUBW\Landespegel Hydrologie\Gewässer\Achkanal\Blaubeuren Achkanal\Blaubeuren Achkanal-Hydrologische Landespegel_cleaned_converted.csv
Converted: Datensätze\UDO-LUBW\Landespegel Hydrologie\Gewässer\Aich\Oberensingen Aich\Oberensingen Aich-Hydrologische Landespegel_cleaned_converted.csv
Converted: Datensätze\UDO-LUBW\Landespegel Hydrologie\Gewässer\Aitrach\Lauben Aitrach\Lauben Aitrach-Hydrologische Landespegel_cleaned_converted.csv
Converted: Datensätze\UDO-LUBW\Landespegel Hydrologie\Gewässer\Alb\Ettlingen Alb\Ettlingen 

### Filtern
In diesem Block werden die konvertierten CSV-Dateien gefiltert, sodass nur noch für die Arbei wesentliche Spalten bestehen bleiben. Übernommen werden für jede Datei die Spalten "Stationsname", "Gewässer", "Datum / Uhrzeit", "Wert in m^3/s". Das daraus resultierende DataFrame wird anschließend als CSV-Datei gespeichert (Endung: "_filtered.csv").

In [14]:
all_converted_files=glob.glob("Datensätze/UDO-LUBW/Landespegel Hydrologie/Gewässer/*/*/*_converted.csv")
def filter_all_converted_files():
    filtered = []
    for p in all_converted_files:
        p = Path(p)
        df = pd.read_csv(p, sep=",", encoding="utf-8")

        df["Wert in m^3/s"] = df["Wert"]
        df_filtered = df[["Stationsname", "Gewässer", "Datum / Uhrzeit", "Wert in m^3/s"]].copy()
        df_filtered["Datum / Uhrzeit"] = pd.to_datetime(df["Datum / Uhrzeit"], errors="coerce")

        new_name = p.name.replace("_cleaned_converted", "_filtered")
        out_path = p.with_name(new_name)

        
        df_filtered.to_csv(out_path, index=False)
        print(f"Filtered: {out_path}")
        filtered.append(out_path)

    return filtered

filtered_files = filter_all_converted_files()
        

print(f"Ready. filtered {len(filtered_files)} files (_filtered.csv)")

Filtered: Datensätze\UDO-LUBW\Landespegel Hydrologie\Gewässer\Ablach\Menningen Ablach\Menningen Ablach-Hydrologische Landespegel_filtered.csv
Filtered: Datensätze\UDO-LUBW\Landespegel Hydrologie\Gewässer\Ach\Blaubeuren Ach\Blaubeuren-Hydrologische Landespegel_filtered.csv
Filtered: Datensätze\UDO-LUBW\Landespegel Hydrologie\Gewässer\Acher\Kappelrodeck Acher\Kappelrodeck Acher-Hydrologische Landespegel_filtered.csv
Filtered: Datensätze\UDO-LUBW\Landespegel Hydrologie\Gewässer\Achkanal\Blaubeuren Achkanal\Blaubeuren Achkanal-Hydrologische Landespegel_filtered.csv
Filtered: Datensätze\UDO-LUBW\Landespegel Hydrologie\Gewässer\Aich\Oberensingen Aich\Oberensingen Aich-Hydrologische Landespegel_filtered.csv
Filtered: Datensätze\UDO-LUBW\Landespegel Hydrologie\Gewässer\Aitrach\Lauben Aitrach\Lauben Aitrach-Hydrologische Landespegel_filtered.csv
Filtered: Datensätze\UDO-LUBW\Landespegel Hydrologie\Gewässer\Alb\Ettlingen Alb\Ettlingen Alb-Hydrologische Landespegel_filtered.csv
Filtered: Datensät

### Kürzen
In diesem Abschnitt werden die gefilterten Dateien gekürzt. Wesentlich ist hier der Zeitraum 2010-01-01 00:00 bis 2024-12-31 23:00. Dies wird ermöglicht, da die Spalte "Datum / Uhrzeit" im datetime-Format vorliegt. Die gekürzten Dateien werden als CSV-Dateien mit der Endung "_filtered_shortened" gespeichert.

In [15]:
# glob all "_filtered.csv" file to shorten them.
all_filtered_files=glob.glob("Datensätze/UDO-LUBW/Landespegel Hydrologie/Gewässer/*/*/*_filtered.csv")

# define 
start_date = pd.Timestamp("2010-01-01 00:00")
end_date = pd.Timestamp("2024-12-31 23:00")

def shorten_files():
    shortened = []

    for p in all_filtered_files:
        p = Path(p)
        
        df = pd.read_csv(p, sep=",", encoding="utf-8")

        df["Datum / Uhrzeit"] = pd.to_datetime(df["Datum / Uhrzeit"], errors="coerce")

        df_shortened = df[(df["Datum / Uhrzeit"] >= start_date) & (df["Datum / Uhrzeit"] <= end_date)]

        new_name = p.name.replace("_filtered", "_filtered_shortened")
        out_path = p.with_name(new_name)

        df_shortened.to_csv(out_path, index=False)
        shortened.append(out_path)

        print(f"Shortened: {out_path.name} ({len(df_shortened)} rows kept, {len(df) - len(df_shortened)} removed)")

    print(f"Ready: {len(shortened)} files from 2010-01-01 00:00 to 2024-12-31 23:00 are created.")
    return shortened


shortened_files = shorten_files()
        
            
        

Shortened: Menningen Ablach-Hydrologische Landespegel_filtered_shortened.csv (131460 rows kept, 94104 removed)
Shortened: Blaubeuren-Hydrologische Landespegel_filtered_shortened.csv (131496 rows kept, 94125 removed)
Shortened: Kappelrodeck Acher-Hydrologische Landespegel_filtered_shortened.csv (131496 rows kept, 94128 removed)
Shortened: Blaubeuren Achkanal-Hydrologische Landespegel_filtered_shortened.csv (131496 rows kept, 59449 removed)
Shortened: Oberensingen Aich-Hydrologische Landespegel_filtered_shortened.csv (131496 rows kept, 94128 removed)
Shortened: Lauben Aitrach-Hydrologische Landespegel_filtered_shortened.csv (131496 rows kept, 94128 removed)
Shortened: Ettlingen Alb-Hydrologische Landespegel_filtered_shortened.csv (131496 rows kept, 94128 removed)
Shortened: Wahlwies Alte Aach-Hydrologische Landespegel_filtered_shortened.csv (131282 rows kept, 94125 removed)
Shortened: Riegel-pumpwerksteg-Hydrologische Landespegel_filtered_shortened.csv (131496 rows kept, 94128 removed)
S

### Analysieren
Dieser Abschnitt untersucht die gekürzten Dateien nach fehlenden stündlichen Zeitstempeln. Jede Datei wird analysiert und in einem zusammenfassenden DataFrame durch eine Zeile repräsentiert. Erfasst werden u. a. Start- und Endzeitpunkt der Zeitreihe, die Dauer (Tage,Jahre), Anzahl der Zeilen, die Anzahl und der Anteil ungültiger Zeitstempel (NaT), sowie die Anzahl der Zeitlücken (Gaps), die Gesamtdauer der fehlenden Stunden, der prozentuale Lückenanteil und die Intervalle der einzelnen Lücken. Zusätzlich werden dieAnzahl und der Anteil fehlender Abflusswerte (NaN) in der Spalte "Wert in m^3/s" ermittelt. Das resultierende DataFrame wird als "timeseries_lengths.csv" gespeichert. Dies dient der späteren Auswahl der Stationen, bei der Abflusszeitreihen mit zu vielen und zu langen zusammenhängenden Lücken ausgeschlossen werden sollen.

In [1]:
from pathlib import Path
import pandas as pd
import glob
import os

all_shortened_files=glob.glob("Datensätze/UDO-LUBW/Landespegel Hydrologie/Gewässer/*/*/*_filtered_shortened.csv")


def find_time_gaps(ts: pd.Series):
    gaps = []
    if ts.size < 2:
        return gaps

    step = pd.Timedelta(hours=1)
    diffs = ts.diff()
    jump_idx = diffs[diffs > step].index

    for i in jump_idx:
        right = ts.loc[i]
        left = ts.loc[ts.index[ts.index.get_loc(i) - 1]]
        diff = right - left
        missing_steps = int(diff // step) - 1
        if missing_steps <= 0:
            continue
        gap_start = left + step
        gap_end = right - step
        missing_hours = float(missing_steps * (step / pd.Timedelta(hours=1)))
        gaps.append((gap_start, gap_end, missing_steps, round(missing_hours, 2)))
    return gaps

def analyze_files():
    analyzed = []
    for p in all_shortened_files:
        p = Path(p)
        
        df = pd.read_csv(p, sep=",", encoding="utf-8")

        if df.empty:
            analyzed.append({
                "Waters": Path(p).parts[-3], "Station": Path(p).parts[-2], "Start": pd.NaT, "End": pd.NaT, 
            "Duration Days": pd.NA, "Duration Years": pd.NA, 
            "Number of rows": 0, "Number of NaTs": 0, "Percentage of NaTs": 0.0,
            "Number of Gaps": 0, "Total missing hours": 0,
            "Gap percentage": 0.0, "Gap intervals": "",
            "Number of NaN values": 0, "Percentage of NaN values": 0.0
            
            })
            print(f"Skipped {p.name} because the file is empty")
            continue
                
        
        waters = df["Gewässer"].iloc[0]
        station = df["Stationsname"].iloc[0]

        num_nan_values = df["Wert in m^3/s"].isna().sum() if "Wert in m^3/s" in df.columns else pd.NA
        percent_nan_values = round(num_nan_values / len(df) * 100, 2) if len(df) > 0 else 0.0

        ts_raw = pd.to_datetime(df["Datum / Uhrzeit"], errors="coerce")

        num_nats = ts_raw.isna().sum()
        percent_nats = round(num_nats / len(ts_raw) * 100,2)

        ts = ts_raw.dropna().sort_values()
        
        if ts.empty:
            start = end = pd.NaT
            duration_days = pd.NA
            duration_years = pd.NA
            num_gaps = 0
            total_missing_hours = 0.0
            gap_percentage = 0.0
            gap_intervals = ""

        else:
            start = ts.iloc[0]
            end = ts.iloc[-1]

            duration_days = int((end - start).days)
            duration_years = round(duration_days / 365.25, 2)
            total_hours = float((end - start) / pd.Timedelta(hours=1))

            gaps = find_time_gaps(ts)
            num_gaps = len(gaps)
            total_missing_hours = sum(g[3] for g in gaps)
            gap_percentage = round((total_missing_hours / total_hours) * 100, 2) if total_hours > 0 else 0.0

            def format(dt):
                return pd.to_datetime(dt).strftime("%Y-%m-%d %H:%M")
            gap_intervals = "; ".join(f"{format(s)}-{format(e)} [Steps={steps}, Hours={hours}]"
                                      for (s, e, steps, hours) in gaps)


       
        analyzed.append({
            "Waters": waters, "Station": station, "Start": start, "End": end, 
            "Duration Days": duration_days, "Duration Years": duration_years, 
            "Number of rows": len(df), "Number of NaTs": num_nats, "Percentage of NaTs": percent_nats,
            "Number of Gaps": num_gaps, "Total missing hours": round(total_missing_hours, 2),
            "Gap percentage": gap_percentage, "Gap intervals": gap_intervals,
            "Number of NaN values": num_nan_values, "Percentage of NaN values": percent_nan_values
            
        })
         
        print(f"Analyzed: {p.name}")
    return pd.DataFrame(analyzed)
    
results = analyze_files()      
results.to_csv("timeseries_lengths.csv", index=False)

print("Results stored in 'timeseries_lengths.csv'")
print(results.head(20))
   
    


Analyzed: Menningen Ablach-Hydrologische Landespegel_filtered_shortened.csv
Analyzed: Blaubeuren-Hydrologische Landespegel_filtered_shortened.csv
Analyzed: Kappelrodeck Acher-Hydrologische Landespegel_filtered_shortened.csv
Analyzed: Blaubeuren Achkanal-Hydrologische Landespegel_filtered_shortened.csv
Analyzed: Oberensingen Aich-Hydrologische Landespegel_filtered_shortened.csv
Analyzed: Lauben Aitrach-Hydrologische Landespegel_filtered_shortened.csv
Analyzed: Ettlingen Alb-Hydrologische Landespegel_filtered_shortened.csv
Analyzed: Wahlwies Alte Aach-Hydrologische Landespegel_filtered_shortened.csv
Analyzed: Riegel-pumpwerksteg-Hydrologische Landespegel_filtered_shortened.csv
Analyzed: Pfäffingen Ammer-Hydrologische Landespegel_filtered_shortened.csv
Analyzed: Bittelschieß Andelsbach-Hydrologische Landespegel_filtered_shortened.csv
Analyzed: Gießen Argen-Hydrologische Landespegel_filtered_shortened.csv
Analyzed: Achstetten Baierzer Rot-Hydrologische Landespegel_filtered_shortened.csv
An

### Auswahl nach fehlenden Werten
In diesem Abschnitt werden nun die Ergebnisse aus der Analyse genutzt (in "timeseries_lengths.csv"). Alle CSV-Dateien werden ausgeschlossen, die leer sind, zusammenhängende Lücken von 24 Stunden oder mehr aufweisen oder einen Lückenanteil von mehr als 5 Prozent besitzen. Darüber hinaus werden nur Zeitreihen berücksichtigt, die den Zeitraum exakt abdecken, das heißt deren erste und letzte Zeitstempel genau dem Start (2010-01-01 00:00) und dem Ende (2024-12-31 23:00) entsprechen.
Die verbleibenden Stationen werden in "timeseries_lengths_filtered.csv" gespeichert.

In [8]:
import re

# Define the start and end of the time series
start_date = pd.Timestamp("2010-01-01 00:00")
end_date = pd.Timestamp("2024-12-31 23:00")

# Load the csv file
df = pd.read_csv("timeseries_lengths.csv")

# Only Files which are not empty
df = df[df["Number of rows"] > 0]

# define the columns
gap_percentage_columns = "Gap percentage" 
intervals_columns = "Gap intervals" 

# make start and end to datetime
df["Start"] = pd.to_datetime(df["Start"], errors="coerce")
df["End"] = pd.to_datetime(df["End"], errors="coerce")

# identify biggest gap
def max_gap_hours(text):
    if pd.isna(text) or not str(text).strip():
        return 0.0

    return max(map(float, re.findall(r"Hours=(\d+(?:\.\d+)?)", str(text))))

df["Max gap hours"] = df[intervals_columns].apply(max_gap_hours) 

# filter
non_empty = df["Number of rows"] > 0
no_long_gap = df["Max gap hours"] < 24
gap_percentage = df[gap_percentage_columns].fillna(0) <= 5

# exact time
exact_window = (df["Start"] == start_date) & (df["End"] == end_date)

# filter and store
filtered = df[non_empty & no_long_gap & gap_percentage & exact_window].copy()
filtered.to_csv("timeseries_lengths_filtered.csv", index=False)

print("timeseries_lengths_filtered is stored")
print(f"{len(filtered)} files fit with the criteria.")
print(filtered.head(10))



timeseries_lengths_filtered is stored
194 files fit with the criteria.
          Waters                        Station      Start  \
1            Ach                 Blaubeuren Ach 2010-01-01   
2          Acher             Kappelrodeck Acher 2010-01-01   
3       Achkanal            Blaubeuren Achkanal 2010-01-01   
4           Aich              Oberensingen Aich 2010-01-01   
5        Aitrach                 Lauben Aitrach 2010-01-01   
6            Alb                  Ettlingen Alb 2010-01-01   
8       Alte Elz  Riegel-Pumpwerksteg  Alte Elz 2010-01-01   
10    Andelsbach        Bittelschieß Andelsbach 2010-01-01   
11         Argen                   Gießen Argen 2010-01-01   
12  Baierzer Rot        Achstetten Baierzer Rot 2010-01-01   

                   End  Duration Days  Duration Years  Number of rows  \
1  2024-12-31 23:00:00         5478.0            15.0          131496   
2  2024-12-31 23:00:00         5478.0            15.0          131496   
3  2024-12-31 23:00:00     

### Auflistung der verbliebenen Fließgewässer
Hier werden die Fließgewässer aufgelistet von denen Stationen verblieben sind.

In [5]:
df = pd.read_csv("timeseries_lengths_filtered.csv")

remaining_waters = sorted(df["Waters"].dropna().unique())

number_waters = len(remaining_waters)

print(f"There are still {number_waters} waters left.\n")
print("List of remaining waters")
for w in remaining_waters:
    print(" -", w)

pd.Series(remaining_waters, name="Waters").to_csv("remaining_waters.csv", index=False)
print("\n Data frame stored in remaining_waters.csv")

There are still 134 waters left.

List of remaining waters
 - Ach
 - Acher
 - Achkanal
 - Aich
 - Aitrach
 - Alb
 - Alte Elz
 - Andelsbach
 - Argen
 - Baierzer Rot
 - Bibers
 - Blau
 - Blinde Rot
 - Bottwar
 - Breg
 - Brehmbach
 - Brenz
 - Brettach
 - Brigach
 - Bronnbachquelle
 - Brotenaubach
 - Buchenbach
 - Bära
 - Bühler
 - Deggenhauser Aach
 - Donau
 - Dürnach
 - Echaz
 - Eger
 - Elta
 - Elz
 - Ennetacher Ablach
 - Enz
 - Erfa
 - Erlenbach
 - Erms
 - Eschach
 - Eyach
 - Eyb
 - Fichtenberger Rot
 - Fils
 - Forbach
 - Gießbach
 - Glatt
 - Glems
 - Große Enz
 - Große Lauter
 - Grünbach
 - Gutach
 - Hasel
 - Haslach
 - Hürbe
 - Itter
 - Jagst
 - Kanzach
 - Kinzig
 - Kirnach
 - Kleine Wiese
 - Kocher
 - Kraichbach
 - Krähenbach
 - Körsch
 - Kürnach
 - Lauchert
 - Lauter
 - Leerausbach
 - Leimbach
 - Lein
 - Leopoldskanal
 - Linach
 - Lindach
 - Lone
 - Mengener Ablach
 - Murg
 - Murr
 - Möhlin
 - Nagold
 - Neckar
 - Neumagen
 - Obere Argen
 - Ohrn
 - Oos
 - Ostrach
 - Pfaffenrieder Bac

### Endgültige Auswahl durch räumliche Verteilung
Dieser Block verwendet die ermittelten Koordinaten der verbliebenen Stationen (in "Koordinaten der Gewässerstationen.csv") und wählt mittels "Farthest Point Sampling" (FPS) insgesamt 25 Stationen aus, die räumlich möglichst gleichmäßig verteilt sind. Die erste Station wird dabei als der am weitesten vom Mittelpunkt entfernte Punkt gewählt. Anschließend werden iterativ jeweils die Stationen gewählt, die den größtmöglichen Abstand zur bisherigen Auswahl aufweisen. Zusätzliche Bedingungen sind dabei, dass maximal zwei Stationen dem selben Fließgewässer zugeordnet sind und maximal neun Stationen aus demselben Naturraum stammen. Die daraus resultierenden Stationen werden in der Datei "Endgültige_Auswahl_der_Gewässerstationen.csv" gespeichert.


In [7]:
from pathlib import Path
import pandas as pd
import glob
import numpy as np
from collections import Counter

# Select Stations with FPS
def select_stations(): 
    selected = []
    x_col="Ostwert (ETRS89)"
    y_col="Nordwert (ETRS89)"
    water_stations = pd.read_csv("Koordinaten der Gewässerstationen.csv", encoding="utf-8").copy()

    points = water_stations[[x_col, y_col]].to_numpy(dtype=float)
    waters = water_stations["Gewässer"].astype(str).to_numpy()
    landscapes = water_stations["Landschaftstyp"].astype(str).to_numpy()
    MAX_STATIONS = 25
    

    centroid = points.mean(axis=0)
    
    # choose the maximum of euclidean distance from the centred mean to select the first station.                      
    starting_point = int(np.argmax(np.sqrt(np.sum((points - centroid)**2, axis=1))))
    selected.append(starting_point)

    distance_minimum = np.sqrt(np.sum((points - points[starting_point])**2, axis=1))

    waters_count = Counter()
    waters_count[waters[starting_point]] += 1
    landscapes_count = Counter()
    landscapes_count[landscapes[starting_point]] += 1

    while (len(selected) < MAX_STATIONS):
        available = np.array([(waters_count[w] < 2) and (landscapes_count[l] < 9) for w,l in zip(waters,landscapes)], dtype=bool)
        available[np.array(selected)] = False

        if not np.any(available):
            break
        i = int(np.argmax(np.where(available, distance_minimum, -np.inf)))
        if waters_count[waters[i]] < 2:
            selected.append(i)
        else:
            continue
        waters_count[waters[i]] += 1
        landscapes_count[landscapes[i]] += 1
        distance = np.sqrt(np.sum((points - points[i])**2, axis=1))
        distance_minimum = np.minimum(distance_minimum, distance)

    selected_water_stations = water_stations.iloc[selected]
    selected_water_stations.to_csv("Endgültige_Auswahl_der_Gewässerstationen.csv", index=False, encoding="utf-8")

    print(f"Selected {len(selected_water_stations)} stations in 'Endgültige_Auswahl_der_Gewässerstationen.csv'.")
    return selected_water_stations


selected_stations = select_stations()
    
    


Selected 25 stations in 'Endgültige_Auswahl_der_Gewässerstationen.csv'.
